# Physical Object: Reservoir

The `Reservoir` class is a fundamental stateful object in the simulation, representing a body of stored water. Its state is determined by the mass balance of inflows and outflows over time.

Functionally similar to the `Lake` model but without an evaporation component, the `Reservoir`'s outflow is not calculated internally. Instead, it is determined by the demand of downstream components and provided to the `step` method via the `action` dictionary.

### State Variables
- `volume` (float): The current volume of water in the reservoir (m^3).
- `water_level` (float): The current water level (m), calculated from volume and surface area.
- `outflow` (float): The outflow from the reservoir for the current step (m^3/s), provided as an input.

### Parameters
- `surface_area` (float): The surface area of the reservoir (m^2), used for calculating water level from volume.

## Simulation Example

The following simulation demonstrates a reservoir's response to variable inflows and outflows. We will provide a cyclical inflow pattern and a steady outflow demand, and observe the resulting changes in the reservoir's volume and water level.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.core.interfaces import State, Parameters

# 1. Define initial state and parameters
initial_volume = 25e6
surface_area = 1.5e6
initial_state = State(volume=initial_volume, water_level=initial_volume/surface_area)
parameters = Parameters(surface_area=surface_area)

# 2. Create a Reservoir instance
reservoir = Reservoir(name="storage_reservoir", initial_state=initial_state, parameters=parameters)

# 3. Simulation settings
dt = 3600 # 1-hour time steps
simulation_duration = 86400 * 4 # 4 days
num_steps = int(simulation_duration / dt)
history = []
inflows = []

# 4. Run the simulation loop
outflow_demand = 200.0 # Constant outflow demand
for t in range(num_steps):
    # Create a cyclical inflow pattern (e.g. from an upstream river)
    inflow = 150 + 100 * np.sin(2 * np.pi * t * dt / 86400)
    inflows.append(inflow)
    reservoir.set_inflow(inflow)
    
    action = {'outflow': outflow_demand}
    
    current_state = reservoir.step(action=action, dt=dt)
    history.append(current_state.copy())

print("Reservoir simulation complete.")

## Results and Visualization

The plots show the reservoir's water level and volume changing in response to the daily inflow cycle against the constant outflow demand.

In [ ]:
# Extract data from history
time_days = [i * dt / 86400 for i in range(num_steps)]
water_levels = [h['water_level'] for h in history]
volumes_Mm3 = [h['volume'] / 1e6 for h in history] # Convert to million cubic meters

# Create a DataFrame
df = pd.DataFrame({
    'Time (days)': time_days,
    'Inflow (m^3/s)': inflows,
    'Water Level (m)': water_levels,
    'Volume (Mm^3)': volumes_Mm3
})

print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Inflow
ax1.plot(df['Time (days)'], df['Inflow (m^3/s)'], label='Inflow', linestyle='--')
ax1.axhline(y=outflow_demand, color='red', linestyle=':', label='Outflow Demand')
ax1.set_ylabel('Flow Rate (m^3/s)')
ax1.set_title('Reservoir Simulation')
ax1.grid(True)
ax1.legend()

# Water Level and Volume
ax2.plot(df['Time (days)'], df['Water Level (m)'], label='Water Level (m)', color='purple')
ax2.set_xlabel('Time (days)')
ax2.set_ylabel('Water Level (m)', color='purple')
ax2.tick_params(axis='y', labelcolor='purple')
ax2.legend(loc='upper left')

ax2_twin = ax2.twinx()
ax2_twin.plot(df['Time (days)'], df['Volume (Mm^3)'], label='Volume (Mm^3)', color='green')
ax2_twin.set_ylabel('Volume (Mm^3)', color='green')
ax2_twin.tick_params(axis='y', labelcolor='green')
ax2_twin.legend(loc='upper right')

plt.tight_layout()
plt.show()